# Citation

Much of the code and examples are copied/modified from 

> Blueprints for Text Analytics Using Python by Jens Albrecht, Sidharth Ramachandran, and Christian Winkler (O'Reilly, 2021), 978-1-492-07408-3.
>

- https://github.com/blueprints-for-text-analytics-python/blueprints-text
- https://github.com/blueprints-for-text-analytics-python/blueprints-text/blob/master/ch08/Topic_Modeling_Clustering.ipynb

---

# Setup

In [ ]:
%run "/code/source/config/notebook_settings.py"

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
from source.library.text_analysis import count_tokens, tf_idf, get_context_from_keyword, count_keywords, count_keywords_by, impurity
from source.library.sklearn_topic_modeling import *
from helpsk.utility import read_pickle

In [ ]:
ngrams_low = XXXXXXXXXXXXXXXX
ngrams_high = YYYYYYYYYYYYYYYY
num_clusters = ZZZZZZZZZZZZZZZZ

In [ ]:
with Timer("Loading Data"):
    path = 'artifacts/data/processed/un-general-debates-paragraphs.pkl'
    paragraphs = pd.read_pickle(path)
    
with Timer("Loading NMF"):
    file = f'artifacts/models/topics/nmf-topics-{num_clusters}-ngrams-{ngrams_low}-{ngrams_high}__vectorizer.pkl'
    nmf_vectorizer = read_pickle(file)

    # file = f'artifacts/models/topics/nmf-topics-10-ngrams-{ngrams_low}-{ngrams_high}__vectors.pkl'
    # nmf_vectors = read_pickle(file)
    
    file = f'artifacts/models/topics/nmf-topics-{num_clusters}-ngrams-{ngrams_low}-{ngrams_high}__model.pkl'  # noqa
    nmf_model = read_pickle(file)

with Timer("Loading LDA"):
    file = f'artifacts/models/topics/lda-topics-{num_clusters}-ngrams-{ngrams_low}-{ngrams_high}__vectorizer.pkl'
    lda_vectorizer = read_pickle(file)

    file = f'artifacts/models/topics/lda-topics-{num_clusters}-ngrams-{ngrams_low}-{ngrams_high}__vectors.pkl'
    lda_vectors = read_pickle(file)
    
    file = f'artifacts/models/topics/lda-topics-{num_clusters}-ngrams-{ngrams_low}-{ngrams_high}__model.pkl'  # noqa
    lda_model = read_pickle(file)

with Timer("Loading K-Means"):
    file = f'artifacts/models/topics/k_means-topics-{num_clusters}-ngrams-{ngrams_low}-{ngrams_high}__vectorizer.pkl'
    k_means_vectorizer = read_pickle(file)

    file = f'artifacts/models/topics/k_means-topics-{num_clusters}-ngrams-{ngrams_low}-{ngrams_high}__vectors.pkl'
    k_means_vectors = read_pickle(file)
    
    file = f'artifacts/models/topics/k_means-topics-{num_clusters}-ngrams-{ngrams_low}-{ngrams_high}__model.pkl'  # noqa
    k_means_model = read_pickle(file)

---

# Exploratory Data Analysis

This section provides a basic exploration of the text and dataset.

## Dataset Summary

In [ ]:
hlp.pandas.numeric_summary(paragraphs)

In [ ]:
hlp.pandas.non_numeric_summary(paragraphs)

In [ ]:
assert not (paragraphs['text'].str.strip() == '').any()

---

# Topic Modeling

## NMF

In [ ]:
nmf_explorer = TopicModelExplorer(model=nmf_model, vectorizer=nmf_vectorizer)

### Overview

In [ ]:
nmf_explorer.plot_topics(
    top_n_tokens=8,
    num_tokens_in_label=2
)

In [ ]:
nmf_explorer.plot_topic_sizes(text_series=paragraphs['text'])

In [ ]:
predicted_topics = nmf_model.transform(X=nmf_vectorizer.transform(paragraphs['text']))
per_document_totals = predicted_topics.sum(axis=1)
ax = pd.Series(per_document_totals).plot(kind='box', vert=False, figsize=(10, 1))
ax.set_title("Distribution Sum of Predicted Values/Topics Per Document")
ax.set_xlabel("Sum of Predicted Values Per Document")
ax.set_yticklabels([])
ax;

---

In [ ]:
nmf_explorer.plot_token('peace')

---

### Trends

In [ ]:
topic_sizes_per_year = nmf_explorer.get_topic_sizes_per_segment(
    df=paragraphs,
    text_column='text',
    segment_column='year',
)
topic_sizes_per_year.head()

In [ ]:
fig = px.area(
    topic_sizes_per_year,
    x="year",
    y="relative_size",
    color="topic_labels",
    title="Topics Over Time",
)
fig.show()

In [ ]:
fig = px.bar(
    topic_sizes_per_year,
    x="year",
    y="relative_size",
    color="topic_labels",
    title="Topics Over Time",
)
fig.show()

In [ ]:
fig = px.line(
    topic_sizes_per_year,
    x="year",
    y="relative_size",
    color="topic_labels",
    title="Topics Over Time",
)
fig.show()

In [ ]:
fig = px.scatter(
    topic_sizes_per_year,
    x="year",
    y="relative_size",
    color="topic_labels",
    trendline="lowess",
    opacity=0.0,
    title="Topics Over Time",
)
fig.show()

---

### Examples

In [ ]:
top_examples = nmf_explorer.extract_top_examples(
    text_series=paragraphs['text'],
    top_n_examples=10
)
top_examples.style.hide(axis='index')

---

## LDA

Neither the book nor the example above uses TF-IDF with LDA, but do not specify why. Both use TF-IDF with NMF and then change to CountVectorizer with LDA


https://stackoverflow.com/questions/44781047/necessary-to-apply-tf-idf-to-new-documents-in-gensim-lda-model/44789327#44789327

> LDA only needs a bag-of-word vector.



In [ ]:
lda_explorer = TopicModelExplorer(model=lda_model, vectorizer=lda_vectorizer)

### Overview

In [ ]:
lda_explorer.plot_topics(
    top_n_tokens=8,
    num_tokens_in_label=2,
    token_separator=' | '
)

In [ ]:
lda_explorer.plot_topic_sizes(
    text_series=paragraphs['text'],
    num_tokens_in_label=3,
    token_separator=' | '
)

In [ ]:
predicted_topics = lda_model.transform(X=lda_vectorizer.transform(paragraphs['text']))
per_document_totals = predicted_topics.sum(axis=1)
ax = pd.Series(per_document_totals).plot(kind='box', vert=False, figsize=(10, 1))
ax.set_title("Distribution Sum of Predicted Values/Topics Per Document")
ax.set_xlabel("Sum of Predicted Values Per Document")
ax.set_yticklabels([])
ax;

---

In [ ]:
lda_explorer.plot_token('peace')

---

### Trends

In [ ]:
topic_sizes_per_year = lda_explorer.get_topic_sizes_per_segment(
    df=paragraphs,
    text_column='text',
    segment_column='year',
)
topic_sizes_per_year.head()

In [ ]:
fig = px.area(
    topic_sizes_per_year,
    x="year",
    y="relative_size",
    color="topic_labels",
    title="Topics Over Time",
)
fig.show()

In [ ]:
fig = px.bar(
    topic_sizes_per_year,
    x="year",
    y="relative_size",
    color="topic_labels",
    title="Topics Over Time",
)
fig.show()

In [ ]:
fig = px.line(
    topic_sizes_per_year,
    x="year",
    y="relative_size",
    color="topic_labels",
    title="Topics Over Time",
)
fig.show()

In [ ]:
fig = px.scatter(
    topic_sizes_per_year,
    x="year",
    y="relative_size",
    color="topic_labels",
    trendline="lowess",
    opacity=0.0,
    title="Topics Over Time",
)
fig.show()

---

### Examples

In [ ]:
top_examples = lda_explorer.extract_top_examples(
    text_series=paragraphs['text'],
    top_n_examples=10
)
top_examples.style.hide(axis='index')

---

In [ ]:
import pyLDAvis.sklearn

# monkey patch vectorizer since sklearn changed the name of the function which pyLDAvis isn't
# expecting
lda_vectorizer.get_feature_names = lda_vectorizer.get_feature_names_out
lda_display = pyLDAvis.sklearn.prepare(lda_model, lda_vectors, lda_vectorizer, sort_topics=False)
# pyLDAvis.display(lda_display)

file_name = f"output/models/topics/lda-{num_clusters}-n-grams-{ngrams_low}-{ngrams_high}.html"
pyLDAvis.save_html(lda_display, file_name)

---

## K-Means

In [ ]:
k_means_explorer = KMeansTopicExplorer(
    model=k_means_model,
    vectorizer=k_means_vectorizer,
    vectors=k_means_vectors
)

### Overview

In [ ]:
k_means_explorer.plot_topics(
    top_n_tokens=8,
    num_tokens_in_label=2
)

In [ ]:
k_means_explorer.plot_topic_sizes(text_series=paragraphs['text'])

### Trends

In [ ]:
topic_sizes_per_year = k_means_explorer.get_topic_sizes_per_segment(
    df=paragraphs,
    text_column='text',
    segment_column='year',
)
topic_sizes_per_year.head()

In [ ]:
fig = px.area(
    topic_sizes_per_year,
    x="year",
    y="relative_size",
    color="topic_labels",
    title="Topics Over Time",
)
fig.show()

In [ ]:
fig = px.bar(
    topic_sizes_per_year,
    x="year",
    y="relative_size",
    color="topic_labels",
    title="Topics Over Time",
)
fig.show()

In [ ]:
fig = px.line(
    topic_sizes_per_year,
    x="year",
    y="relative_size",
    color="topic_labels",
    title="Topics Over Time",
)
fig.show()

In [ ]:
fig = px.scatter(
    topic_sizes_per_year,
    x="year",
    y="relative_size",
    color="topic_labels",
    trendline="lowess",
    opacity=0.0,
    title="Topics Over Time",
)
fig.show()

---

### Examples

In [ ]:
top_examples = k_means_explorer.extract_top_examples(
    text_series=paragraphs['text'],
    top_n_examples=10
)
top_examples.style.hide(axis='index')

---